In [1]:
!pip install gensim --user
!pip install textblob

In [2]:
import pandas as pd
import numpy as np
import re

#Timing
from tqdm import tqdm

#Word Embeddings
import nltk
from textblob import TextBlob
from gensim.models import Word2Vec
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

#Models
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to C:\Users\Cameron
[nltk_data]     Milne\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to C:\Users\Cameron
[nltk_data]     Milne\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

## EDA

In [3]:
scored_tweets = pd.read_csv('scored_tweets.csv')

In [4]:
scored_tweets.columns

Index(['created_at', 'text', 'username', 'screen_name', 'verified',
       'followers_count', 'TOXICITY', 'INSULT', 'PROFANITY', 'THREAT',
       'SEXUALLY_EXPLICIT'],
      dtype='object')

In [5]:
scored_tweets.corr()

,TOXICITY,INSULT,PROFANITY,THREAT,SEXUALLY_EXPLICIT
TOXICITY,1.000000,0.945739,0.921274,0.572428,0.700610
INSULT,0.945739,1.000000,0.827742,0.464412,0.542923
PROFANITY,0.921274,0.827742,1.000000,0.442225,0.758937
THREAT,0.572428,0.464412,0.442225,1.000000,0.425812
SEXUALLY_EXPLICIT,0.700610,0.542923,0.758937,0.425812,1.000000


In [6]:
scored_tweets.head(3)

,created_at,text,username,screen_name,verified,followers_count,TOXICITY,INSULT,PROFANITY,THREAT,SEXUALLY_EXPLICIT
0,created_at,text,username,screen_name,verified,followers_count,0.042518,0.015740,0.037332,0.043029,0.040426
1,Thu Dec 02 15:39:21 +0000 2021,This needs to become a real thing. I wanna see...,GhostFedoraTWEET,GhostyRBLX,False,28,0.552407,0.666591,0.313011,0.431656,0.100750
2,Thu Dec 02 15:39:21 +0000 2021,@ENHYPEN_members love u more jake,aii.laa,jayenaii,False,85,0.084299,0.061555,0.109649,0.069504,0.119767


## New Preprocessing

In [7]:
scored_tweets.dropna(subset=['text'], inplace=True)

In [8]:
from nltk.tokenize import sent_tokenize, word_tokenize

#tokenizes every tweet
tokenizer = nltk.RegexpTokenizer(r"\w+")
scored_tweets['processed'] = scored_tweets['text'].apply(tokenizer.tokenize)
scored_tweets['processed']

0                                                  [text]
1       [This, needs, to, become, a, real, thing, I, w...
2                  [ENHYPEN_members, love, u, more, jake]
3       [ENHYPEN_members, i, hope, you, ll, get, a, go...
4       [Mal_DuBois, It, was, dragged, across, on, a, ...
                              ...                        
1957    [Nurseries, in, Glasgow, Come, along, to, a, t...
1958    [jimin, is, a, real, life, angel, https, t, co...
1959    [8Ball, Who, Want, it, 600, 00, If, You, WIN, ...
1960               [I, no, longer, have, time, to, waste]
1961    [BugCatNoms, Awwwwww, Puppet, Bugalo, Cat, Tha...
Name: processed, Length: 1962, dtype: object

In [9]:
from nltk.corpus import stopwords 

#Widely used stopword library
stop_words = set(stopwords.words('english'))

#gets rid of numbers
stop_words.update([str(x) for x in np.arange(10)])

#apply to tokens
scored_tweets['processed'] = scored_tweets.processed.apply(lambda x: [i for i in x if not i in stop_words])
scored_tweets['processed']

0                                                  [text]
1       [This, needs, become, real, thing, I, wanna, s...
2                        [ENHYPEN_members, love, u, jake]
3       [ENHYPEN_members, hope, get, good, rest, fight...
4               [Mal_DuBois, It, dragged, across, dinghy]
                              ...                        
1957    [Nurseries, Glasgow, Come, along, taster, sess...
1958    [jimin, real, life, angel, https, co, 1LsoCrsG1I]
1959    [8Ball, Who, Want, 600, 00, If, You, WIN, If, ...
1960                             [I, longer, time, waste]
1961    [BugCatNoms, Awwwwww, Puppet, Bugalo, Cat, Tha...
Name: processed, Length: 1962, dtype: object

In [10]:
scored_tweets['processed'] = [' '.join(l) for l in scored_tweets['processed']]
scored_tweets['processed']

0                                                    text
1       This needs become real thing I wanna see full ...
2                             ENHYPEN_members love u jake
3       ENHYPEN_members hope get good rest fighting to...
4                     Mal_DuBois It dragged across dinghy
                              ...                        
1957    Nurseries Glasgow Come along taster session fi...
1958            jimin real life angel https co 1LsoCrsG1I
1959    8Ball Who Want 600 00 If You WIN If I Win TRYI...
1960                                  I longer time waste
1961    BugCatNoms Awwwwww Puppet Bugalo Cat Thank u n...
Name: processed, Length: 1962, dtype: object

## Models

In [11]:
!pip install lightgbm
!pip install hyperopt

In [12]:
def split_data(df=scored_tweets, feature='processed', label='TOXICITY'):
    from sklearn.model_selection import train_test_split
    
    train_data, test_data = train_test_split(df, test_size=0.25, random_state=7)
    X_train, X_test = np.asarray(train_data['processed']), np.asarray(test_data['processed'])
    y_train, y_test = np.asarray(train_data[label]), np.asarray(test_data[label])
    
    return X_train, X_test, y_train, y_test

In [13]:
def TFIDF(X_train, X_test):
    from sklearn.feature_extraction.text import TfidfVectorizer

    vectorizer = TfidfVectorizer()
    train_tfidf = vectorizer.fit_transform(X_train)
    test_tfidf = vectorizer.transform(X_test)
    
    return train_tfidf, test_tfidf    

In [14]:
def LinearRegression(feature_category):
    from sklearn.linear_model import LinearRegression
    from sklearn.metrics import mean_absolute_error
    from sklearn.metrics import r2_score
    from sklearn.metrics import accuracy_score
    
    X_train, X_test, y_train, y_test = split_data(label=feature_category)
    train_tfidf, test_tfidf = TFIDF(X_train, X_test)

    model = LinearRegression().fit(train_tfidf, y_train)
    y_pred = model.predict(test_tfidf)
    
    mae = round(mean_absolute_error(y_pred, y_test), 4)
    r2 = round(r2_score(y_pred, y_test), 4)
    
    return (r2, mae, model)

In [15]:
def LightGBM(feature_category):
    import lightgbm as lgb
    from sklearn.metrics import mean_absolute_error
    from sklearn.metrics import r2_score
    
    X_train, X_test, y_train, y_test = split_data(label=feature_category)
    train_tfidf, test_tfidf = TFIDF(X_train, X_test)
    
    hyper_params = {
    'task': 'train',
    'boosting_type': 'gbdt',
    'objective': 'regression',
    'metric': ['l1','l2'],
    'learning_rate': 0.005,
    'feature_fraction': 0.9,
    'bagging_fraction': 0.7,
    'bagging_freq': 10,
    'verbose': 0,
    "max_depth": 8,
    "num_leaves": 128,  
    "max_bin": 512,
    "num_iterations": 100000
    }
    
    gbm = lgb.LGBMRegressor(**hyper_params)
    gbm.fit(train_tfidf, y_train, eval_set=[(test_tfidf, y_test)], eval_metric='l1', early_stopping_rounds=1000)
    y_pred = gbm.predict(test_tfidf, num_iteration=gbm.best_iteration_)
    
    mae = round(mean_absolute_error(y_pred, y_test), 4)
    r2 = round(r2_score(y_pred, y_test), 4)
    
    return (r2, mae, gbm)

In [16]:
def SVR(feature_category):
    from sklearn.svm import SVR
    from sklearn.metrics import mean_absolute_error
    from sklearn.metrics import r2_score
    
    X_train, X_test, y_train, y_test = split_data(label=feature_category)
    train_tfidf, test_tfidf = TFIDF(X_train, X_test)
    
    svr = SVR(kernel='linear').fit(train_tfidf, y_train)
    y_pred = svr.predict(test_tfidf)
    
    mae = round(mean_absolute_error(y_pred, y_test), 4)
    r2 = round(r2_score(y_pred, y_test), 4) 
    
    return (r2, mae, svr)

In [17]:
def ElasticNet(feature_category):
    from sklearn.linear_model import ElasticNet
    from sklearn.metrics import mean_absolute_error
    from sklearn.metrics import r2_score
    
    X_train, X_test, y_train, y_test = split_data(label=feature_category)
    train_tfidf, test_tfidf = TFIDF(X_train, X_test)
    
    model = ElasticNet(random_state=0).fit(train_tfidf, y_train)
    y_pred = model.predict(test_tfidf)
    
    mae = round(mean_absolute_error(y_pred, y_test), 4)
    r2 = round(r2_score(y_pred, y_test), 4) 
    
    return (r2, mae, model)

### Toxicity

In [18]:
category = 'TOXICITY'

lr_results = LinearRegression(category)
gbm_results = LightGBM(category)
svr_results = SVR(category)
en_results = ElasticNet(category)

C:\ProgramData\Anaconda3\lib\site-packages\lightgbm\sklearn.py:726: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
C:\ProgramData\Anaconda3\lib\site-packages\lightgbm\engine.py:177: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")


[LightGBM] [Warning] feature_fraction is set=0.9, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.9
[LightGBM] [Warning] bagging_fraction is set=0.7, subsample=1.0 will be ignored. Current value: bagging_fraction=0.7
[LightGBM] [Warning] bagging_freq is set=10, subsample_freq=0 will be ignored. Current value: bagging_freq=10
[LightGBM] [Warning] feature_fraction is set=0.9, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.9
[LightGBM] [Warning] bagging_fraction is set=0.7, subsample=1.0 will be ignored. Current value: bagging_fraction=0.7
[LightGBM] [Warning] bagging_freq is set=10, subsample_freq=0 will be ignored. Current value: bagging_freq=10
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.005457 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Warning] feature_fraction is set=0.9, colsample_bytree=1.

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[56]	valid_0's l1: 0.15128	valid_0's l2: 0.0422617
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[57]	valid_0's l1: 0.15122	valid_0's l2: 0.0422299
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[58]	valid_0's l1: 0.151149	valid_0's l2: 0.0421945
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[59]	valid_0's l1: 0.15108	valid_0's l2: 0.0421588
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[60]	valid_0's l1: 0.151016	valid_0's l2: 0.042122
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[61]	valid_0's l1: 0.150965	valid_0's l2: 0.0421012
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[62]	valid_0's l1: 0.150917	valid_0's l2: 0.0420823
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[63]	valid_0's l1: 0.150856	valid_0's l2

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[161]	valid_0's l1: 0.148512	valid_0's l2: 0.0407365
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[162]	valid_0's l1: 0.148485	valid_0's l2: 0.0407185
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[163]	valid_0's l1: 0.14846	valid_0's l2: 0.0407046
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[164]	valid_0's l1: 0.148435	valid_0's l2: 0.0406909
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[165]	valid_0's l1: 0.148421	valid_0's l2: 0.0406799
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[166]	valid_0's l1: 0.148395	valid_0's l2: 0.0406621
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[167]	valid_0's l1: 0.148368	valid_0's l2: 0.0406431
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[168]	valid_0's l1: 0.148347	v

[273]	valid_0's l1: 0.147308	valid_0's l2: 0.0398025
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[274]	valid_0's l1: 0.147301	valid_0's l2: 0.0397972
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[275]	valid_0's l1: 0.147274	valid_0's l2: 0.0397887
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[276]	valid_0's l1: 0.147256	valid_0's l2: 0.039785
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[277]	valid_0's l1: 0.147243	valid_0's l2: 0.0397818
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[278]	valid_0's l1: 0.147239	valid_0's l2: 0.0397742
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[279]	valid_0's l1: 0.14723	valid_0's l2: 0.0397709
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[280]	valid_0's l1: 0.147209	valid_0's l2: 0.0397648
[LightGBM] [Warning] No further splits with positive 

...

[1295]	valid_0's l1: 0.145451	valid_0's l2: 0.039538
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[1296]	valid_0's l1: 0.145453	valid_0's l2: 0.0395421
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[1297]	valid_0's l1: 0.145456	valid_0's l2: 0.0395463
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[1298]	valid_0's l1: 0.145454	valid_0's l2: 0.0395469
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[1299]	valid_0's l1: 0.145456	valid_0's l2: 0.0395511
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[1300]	valid_0's l1: 0.145459	valid_0's l2: 0.0395554
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[1301]	valid_0's l1: 0.145444	valid_0's l2: 0.0395508
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[1302]	valid_0's l1: 0.145429	valid_0's l2: 0.0395463
[LightGBM] [Warning] No further splits with 

[1364]	valid_0's l1: 0.145612	valid_0's l2: 0.0396396
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[1365]	valid_0's l1: 0.145631	valid_0's l2: 0.0396454
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[1366]	valid_0's l1: 0.145636	valid_0's l2: 0.0396497
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[1367]	valid_0's l1: 0.145649	valid_0's l2: 0.0396544
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[1368]	valid_0's l1: 0.145663	valid_0's l2: 0.0396592
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[1369]	valid_0's l1: 0.145679	valid_0's l2: 0.0396637
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[1370]	valid_0's l1: 0.14569	valid_0's l2: 0.0396677
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[1371]	valid_0's l1: 0.145697	valid_0's l2: 0.0396664
[LightGBM] [Warning] No further splits with 

[1484]	valid_0's l1: 0.145952	valid_0's l2: 0.0396993
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[1485]	valid_0's l1: 0.145947	valid_0's l2: 0.0396984
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[1486]	valid_0's l1: 0.145942	valid_0's l2: 0.0396975
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[1487]	valid_0's l1: 0.145935	valid_0's l2: 0.0396958
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[1488]	valid_0's l1: 0.14593	valid_0's l2: 0.039695
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[1489]	valid_0's l1: 0.145923	valid_0's l2: 0.0396933
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[1490]	valid_0's l1: 0.145922	valid_0's l2: 0.0396926
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[1491]	valid_0's l1: 0.145922	valid_0's l2: 0.0396923
[LightGBM] [Warning] No further splits with p

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[1614]	valid_0's l1: 0.145911	valid_0's l2: 0.0396631
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[1615]	valid_0's l1: 0.145919	valid_0's l2: 0.0396673
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[1616]	valid_0's l1: 0.145927	valid_0's l2: 0.0396715
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[1617]	valid_0's l1: 0.145926	valid_0's l2: 0.0396693
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[1618]	valid_0's l1: 0.145934	valid_0's l2: 0.0396735
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[1619]	valid_0's l1: 0.145943	valid_0's l2: 0.0396771
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[1620]	valid_0's l1: 0.14595	valid_0's l2: 0.0396811
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[1621]	valid_0's l1: 0.

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[1735]	valid_0's l1: 0.146013	valid_0's l2: 0.0397718
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[1736]	valid_0's l1: 0.146011	valid_0's l2: 0.0397696
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[1737]	valid_0's l1: 0.146012	valid_0's l2: 0.0397695
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[1738]	valid_0's l1: 0.14602	valid_0's l2: 0.0397707
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[1739]	valid_0's l1: 0.14602	valid_0's l2: 0.0397706
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[1740]	valid_0's l1: 0.14602	valid_0's l2: 0.0397704
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[1741]	valid_0's l1: 0.146016	valid_0's l2: 0.0397666
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[1742]	valid_0's l1: 0.14

In [19]:
print("Toxicity Results")
print("================")
print("[r2_score, mean_absolute_error, model]")
print("================")
for model in [lr_results, gbm_results, svr_results, en_results]:
    print(model)

Toxicity Results
[r2_score, mean_absolute_error, model]
(-0.4175, 0.1149, LinearRegression())
(-11.9107, 0.1451, LGBMRegressor(bagging_fraction=0.7, bagging_freq=10, feature_fraction=0.9,
              learning_rate=0.005, max_bin=512, max_depth=8,
              metric=['l1', 'l2'], num_iterations=100000, num_leaves=128,
              objective='regression', task='train', verbose=0))
(-1.597, 0.1341, SVR(kernel='linear'))
(-1.408699114373207e+31, 0.1532, ElasticNet(random_state=0))


### Profanity

In [20]:
category = 'PROFANITY'

lr_results = LinearRegression(category)
gbm_results = LightGBM(category)
svr_results = SVR(category)
en_results = ElasticNet(category)

C:\ProgramData\Anaconda3\lib\site-packages\lightgbm\sklearn.py:726: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
C:\ProgramData\Anaconda3\lib\site-packages\lightgbm\engine.py:177: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")


[LightGBM] [Warning] feature_fraction is set=0.9, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.9
[LightGBM] [Warning] bagging_fraction is set=0.7, subsample=1.0 will be ignored. Current value: bagging_fraction=0.7
[LightGBM] [Warning] bagging_freq is set=10, subsample_freq=0 will be ignored. Current value: bagging_freq=10
[LightGBM] [Warning] feature_fraction is set=0.9, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.9
[LightGBM] [Warning] bagging_fraction is set=0.7, subsample=1.0 will be ignored. Current value: bagging_fraction=0.7
[LightGBM] [Warning] bagging_freq is set=10, subsample_freq=0 will be ignored. Current value: bagging_freq=10
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.003049 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Warning] feature_fraction is set=0.9, colsample_bytree=1.

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[86]	valid_0's l1: 0.12951	valid_0's l2: 0.0387032
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[87]	valid_0's l1: 0.129492	valid_0's l2: 0.0386895
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[88]	valid_0's l1: 0.129463	valid_0's l2: 0.0386763
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[89]	valid_0's l1: 0.129439	valid_0's l2: 0.0386634
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[90]	valid_0's l1: 0.129415	valid_0's l2: 0.0386506
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[91]	valid_0's l1: 0.129384	valid_0's l2: 0.0386374
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[92]	valid_0's l1: 0.129354	valid_0's l2: 0.0386236
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[93]	valid_0's l1: 0.129339	valid_0's

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[190]	valid_0's l1: 0.128322	valid_0's l2: 0.037822
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[191]	valid_0's l1: 0.128324	valid_0's l2: 0.0378147
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[192]	valid_0's l1: 0.128325	valid_0's l2: 0.0378098
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[193]	valid_0's l1: 0.128325	valid_0's l2: 0.0378029
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[194]	valid_0's l1: 0.128329	valid_0's l2: 0.0377965
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[195]	valid_0's l1: 0.128317	valid_0's l2: 0.0377897
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[196]	valid_0's l1: 0.128321	valid_0's l2: 0.0377835
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[197]	valid_0's l1: 0.128334	v

[277]	valid_0's l1: 0.12839	valid_0's l2: 0.0375456
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[278]	valid_0's l1: 0.128399	valid_0's l2: 0.0375452
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[279]	valid_0's l1: 0.128382	valid_0's l2: 0.0375417
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[280]	valid_0's l1: 0.128364	valid_0's l2: 0.0375383
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[281]	valid_0's l1: 0.128355	valid_0's l2: 0.0375346
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[282]	valid_0's l1: 0.128355	valid_0's l2: 0.0375322
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[283]	valid_0's l1: 0.128343	valid_0's l2: 0.0375264
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[284]	valid_0's l1: 0.128343	valid_0's l2: 0.0375241
[LightGBM] [Warning] No further splits with positive

...

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[930]	valid_0's l1: 0.128196	valid_0's l2: 0.0378951
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[931]	valid_0's l1: 0.1282	valid_0's l2: 0.0378917
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[932]	valid_0's l1: 0.128204	valid_0's l2: 0.0378905
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[933]	valid_0's l1: 0.128206	valid_0's l2: 0.0378864
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[934]	valid_0's l1: 0.128208	valid_0's l2: 0.0378824
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[935]	valid_0's l1: 0.128215	valid_0's l2: 0.0378802
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[936]	valid_0's l1: 0.128218	valid_0's l2: 0.0378766
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[937]	valid_0's l1: 0.128227	va

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[1062]	valid_0's l1: 0.128941	valid_0's l2: 0.0380919
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[1063]	valid_0's l1: 0.128958	valid_0's l2: 0.0380969
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[1064]	valid_0's l1: 0.128974	valid_0's l2: 0.038102
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[1065]	valid_0's l1: 0.128993	valid_0's l2: 0.0381066
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[1066]	valid_0's l1: 0.129011	valid_0's l2: 0.0381117
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[1067]	valid_0's l1: 0.129026	valid_0's l2: 0.0381152
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[1068]	valid_0's l1: 0.129043	valid_0's l2: 0.0381203
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[1069]	valid_0's l1: 0.

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[1128]	valid_0's l1: 0.129616	valid_0's l2: 0.0383374
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[1129]	valid_0's l1: 0.129623	valid_0's l2: 0.0383413
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[1130]	valid_0's l1: 0.129633	valid_0's l2: 0.0383469
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[1131]	valid_0's l1: 0.12963	valid_0's l2: 0.0383496
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[1132]	valid_0's l1: 0.12963	valid_0's l2: 0.0383522
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[1133]	valid_0's l1: 0.129632	valid_0's l2: 0.0383561
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[1134]	valid_0's l1: 0.129636	valid_0's l2: 0.0383619
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[1135]	valid_0's l1: 0.1

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[1193]	valid_0's l1: 0.129577	valid_0's l2: 0.0383645
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[1194]	valid_0's l1: 0.129579	valid_0's l2: 0.0383619
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[1195]	valid_0's l1: 0.129574	valid_0's l2: 0.0383564
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[1196]	valid_0's l1: 0.129573	valid_0's l2: 0.038352
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[1197]	valid_0's l1: 0.129568	valid_0's l2: 0.0383467
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[1198]	valid_0's l1: 0.129569	valid_0's l2: 0.0383442
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[1199]	valid_0's l1: 0.129576	valid_0's l2: 0.0383423
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[1200]	valid_0's l1: 0.

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[1257]	valid_0's l1: 0.12922	valid_0's l2: 0.0383253
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[1258]	valid_0's l1: 0.129218	valid_0's l2: 0.038323
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[1259]	valid_0's l1: 0.129215	valid_0's l2: 0.038322
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[1260]	valid_0's l1: 0.129214	valid_0's l2: 0.0383198
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[1261]	valid_0's l1: 0.129209	valid_0's l2: 0.0383194
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[1262]	valid_0's l1: 0.129206	valid_0's l2: 0.0383181
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[1263]	valid_0's l1: 0.129199	valid_0's l2: 0.0383153
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[1264]	valid_0's l1: 0.12

In [21]:
print("Profanity Results")
print("================")
print("[r2_score, mean_absolute_error, model]")
print("================")
for model in [lr_results, gbm_results, svr_results, en_results]:
    print(model)

Profanity Results
[r2_score, mean_absolute_error, model]
(-0.5004, 0.1062, LinearRegression())
(-39.1523, 0.1281, LGBMRegressor(bagging_fraction=0.7, bagging_freq=10, feature_fraction=0.9,
              learning_rate=0.005, max_bin=512, max_depth=8,
              metric=['l1', 'l2'], num_iterations=100000, num_leaves=128,
              objective='regression', task='train', verbose=0))
(-1.8409, 0.1287, SVR(kernel='linear'))
(-5.170404951616488e+31, 0.1313, ElasticNet(random_state=0))


### Sexually Explicit

In [22]:
category = 'SEXUALLY_EXPLICIT'

lr_results = LinearRegression(category)
gbm_results = LightGBM(category)
svr_results = SVR(category)
en_results = ElasticNet(category)

C:\ProgramData\Anaconda3\lib\site-packages\lightgbm\sklearn.py:726: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
C:\ProgramData\Anaconda3\lib\site-packages\lightgbm\engine.py:177: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")


[LightGBM] [Warning] feature_fraction is set=0.9, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.9
[LightGBM] [Warning] bagging_fraction is set=0.7, subsample=1.0 will be ignored. Current value: bagging_fraction=0.7
[LightGBM] [Warning] bagging_freq is set=10, subsample_freq=0 will be ignored. Current value: bagging_freq=10
[LightGBM] [Warning] feature_fraction is set=0.9, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.9
[LightGBM] [Warning] bagging_fraction is set=0.7, subsample=1.0 will be ignored. Current value: bagging_fraction=0.7
[LightGBM] [Warning] bagging_freq is set=10, subsample_freq=0 will be ignored. Current value: bagging_freq=10
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.004508 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Warning] feature_fraction is set=0.9, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.9
[LightGBM

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[85]	valid_0's l1: 0.106574	valid_0's l2: 0.0285565
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[86]	valid_0's l1: 0.106574	valid_0's l2: 0.0285494
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[87]	valid_0's l1: 0.10657	valid_0's l2: 0.0285433
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[88]	valid_0's l1: 0.106575	valid_0's l2: 0.0285375
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[89]	valid_0's l1: 0.106574	valid_0's l2: 0.0285302
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[90]	valid_0's l1: 0.106572	valid_0's l2: 0.0285228
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[91]	valid_0's l1: 0.106582	valid_0's l2: 0.0285219
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[92]	valid_0's l1: 0.106592	valid_0's

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[211]	valid_0's l1: 0.106768	valid_0's l2: 0.0285237
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[212]	valid_0's l1: 0.106765	valid_0's l2: 0.028521
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[213]	valid_0's l1: 0.106761	valid_0's l2: 0.0285183
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[214]	valid_0's l1: 0.106758	valid_0's l2: 0.0285157
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[215]	valid_0's l1: 0.106748	valid_0's l2: 0.0285129
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[216]	valid_0's l1: 0.106738	valid_0's l2: 0.0285101
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[217]	valid_0's l1: 0.106755	valid_0's l2: 0.0285146
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[218]	valid_0's l1: 0.106745	v

[335]	valid_0's l1: 0.106635	valid_0's l2: 0.0285785
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[336]	valid_0's l1: 0.106626	valid_0's l2: 0.0285795
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[337]	valid_0's l1: 0.106606	valid_0's l2: 0.028578
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[338]	valid_0's l1: 0.106605	valid_0's l2: 0.0285815
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[339]	valid_0's l1: 0.106595	valid_0's l2: 0.0285824
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[340]	valid_0's l1: 0.106586	valid_0's l2: 0.0285835
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[341]	valid_0's l1: 0.106583	valid_0's l2: 0.0285826
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[342]	valid_0's l1: 0.106588	valid_0's l2: 0.0285832
[LightGBM] [Warning] No further splits with positive

...

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[533]	valid_0's l1: 0.106667	valid_0's l2: 0.0288
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[534]	valid_0's l1: 0.106667	valid_0's l2: 0.0287991
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[535]	valid_0's l1: 0.106672	valid_0's l2: 0.0287962
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[536]	valid_0's l1: 0.106683	valid_0's l2: 0.0287945
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[537]	valid_0's l1: 0.106694	valid_0's l2: 0.0287928
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[538]	valid_0's l1: 0.10669	valid_0's l2: 0.0287913
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[539]	valid_0's l1: 0.106696	valid_0's l2: 0.0287903
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[540]	valid_0's l1: 0.106707	vali

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[598]	valid_0's l1: 0.106986	valid_0's l2: 0.0288934
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[599]	valid_0's l1: 0.106992	valid_0's l2: 0.0288976
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[600]	valid_0's l1: 0.106987	valid_0's l2: 0.0288995
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[601]	valid_0's l1: 0.106985	valid_0's l2: 0.0288994
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[602]	valid_0's l1: 0.106984	valid_0's l2: 0.0288988
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[603]	valid_0's l1: 0.106976	valid_0's l2: 0.0288963
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[604]	valid_0's l1: 0.106982	valid_0's l2: 0.0288967
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[605]	valid_0's l1: 0.106976	

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[722]	valid_0's l1: 0.10775	valid_0's l2: 0.029017
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[723]	valid_0's l1: 0.107753	valid_0's l2: 0.0290178
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[724]	valid_0's l1: 0.107759	valid_0's l2: 0.0290187
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[725]	valid_0's l1: 0.107767	valid_0's l2: 0.0290211
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[726]	valid_0's l1: 0.107772	valid_0's l2: 0.029021
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[727]	valid_0's l1: 0.107775	valid_0's l2: 0.029022
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[728]	valid_0's l1: 0.107778	valid_0's l2: 0.029023
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[729]	valid_0's l1: 0.107781	valid

[847]	valid_0's l1: 0.108261	valid_0's l2: 0.0292552
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[848]	valid_0's l1: 0.108256	valid_0's l2: 0.0292542
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[849]	valid_0's l1: 0.108252	valid_0's l2: 0.0292527
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[850]	valid_0's l1: 0.10825	valid_0's l2: 0.0292524
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[851]	valid_0's l1: 0.108248	valid_0's l2: 0.0292534
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[852]	valid_0's l1: 0.108246	valid_0's l2: 0.0292541
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[853]	valid_0's l1: 0.108244	valid_0's l2: 0.0292551
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[854]	valid_0's l1: 0.108242	valid_0's l2: 0.0292561
[LightGBM] [Warning] No further splits with positive

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[966]	valid_0's l1: 0.108353	valid_0's l2: 0.0294186
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[967]	valid_0's l1: 0.108359	valid_0's l2: 0.029419
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[968]	valid_0's l1: 0.108372	valid_0's l2: 0.0294221
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[969]	valid_0's l1: 0.10838	valid_0's l2: 0.0294231
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[970]	valid_0's l1: 0.10839	valid_0's l2: 0.0294249
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[971]	valid_0's l1: 0.108407	valid_0's l2: 0.029426
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[972]	valid_0's l1: 0.108413	valid_0's l2: 0.0294248
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[973]	valid_0's l1: 0.108432	vali

In [23]:
print("Sexually Explicit Results")
print("================")
print("[r2_score, mean_absolute_error, model]")
print("================")
for model in [lr_results, gbm_results, svr_results, en_results]:
    print(model)

Sexually Explicit Results
[r2_score, mean_absolute_error, model]
(-0.3737, 0.0969, LinearRegression())
(-118063.8676, 0.1061, LGBMRegressor(bagging_fraction=0.7, bagging_freq=10, feature_fraction=0.9,
              learning_rate=0.005, max_bin=512, max_depth=8,
              metric=['l1', 'l2'], num_iterations=100000, num_leaves=128,
              objective='regression', task='train', verbose=0))
(-1.8407, 0.119, SVR(kernel='linear'))
(-3.7488045902794363e+31, 0.1062, ElasticNet(random_state=0))
